<a href="https://colab.research.google.com/github/UdaraChamidu/Generative-AI/blob/main/Conversational_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Eye Disease Chat Bot** with Langchain and OpenAI LLM

In [85]:
!pip install langchain -qU
!pip install langchain-openai -qU # LLM and Embedding model
!pip install langchain-chroma -qU # vector database
!pip install langchain_community -qU

In [86]:
import os
from google.colab import userdata

# Initialize OpenAI LLM

In [87]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model (language model)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0 # randomness of the response
)

# Initialize Embedding Model

In [88]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

### Load PDF Document

In [89]:
!pip install pypdf -qU

In [90]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("Kanski’s clinical ophthalmology _ a systematic approach.pdf")

docs = loader.load() # document object (page_content and metadata)

In [91]:
len(docs)

897

In [92]:
docs[4]

Document(metadata={'producer': '3-Heights(TM) PDF Producer 4.4.43.3 (http://www.pdf-tools.com); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'creator': 'Elsevier', 'creationdate': '2015-05-13T17:49:48+07:00', 'author': 'Brad Bowling', 'moddate': '2015-07-05T16:31:24+09:30', 'subject': "Kanski's Clinical Ophthalmology, Eighth Edition (2016) ii. doi:10.1016/B978-0-7020-5572-0.00025-8", 'title': "Kanski's Clinical Ophthalmology, Eighth Edition (2016)", 'source': 'Kanski’s clinical ophthalmology _ a systematic approach.pdf', 'total_pages': 897, 'page': 4, 'page_label': 'iii'}, page_content="S\nKanski’s Clinical \nOphthalmology\nA SYSTEMATIC APPROACH\nEIGHTH EDITION\nBrad Bowling  \nFRCSEd(Ophth), FRCOphth, FRANZCO\nOphthalmologist\nSydney\nNew South Wales\nAustralia\nFor additional online content visit expertconsult\nBowling_Title page_main.indd   3 2015-02-25   18:35:53\n7R\x03SURWHFW\x03WKH\x03ULJKWV\x03RI\x03WKH\x03DXWKRU\x0bV\x0c\x03DQG\x03SXEOLVKHU\x03ZH\x03LQIRUP\x03 \\RX\x03WKDW

### Split Documents into Chunks

In [93]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
# create splitter object
splits = text_splitter.split_documents(docs)

In [94]:
len(splits)

6524

In [95]:
splits[0]

Document(metadata={'producer': '3-Heights(TM) PDF Producer 4.4.43.3 (http://www.pdf-tools.com); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'creator': 'Elsevier', 'creationdate': '2015-05-13T17:49:48+07:00', 'author': 'Brad Bowling', 'moddate': '2015-07-05T16:31:24+09:30', 'subject': "Kanski's Clinical Ophthalmology, Eighth Edition (2016) ii. doi:10.1016/B978-0-7020-5572-0.00025-8", 'title': "Kanski's Clinical Ophthalmology, Eighth Edition (2016)", 'source': 'Kanski’s clinical ophthalmology _ a systematic approach.pdf', 'total_pages': 897, 'page': 2, 'page_label': 'i'}, page_content="S\nKanski’s Clinical \nOphthalmology\nA SYSTEMATIC APPROACH\nBowling_Half title page_main.indd   1 2015-02-25   18:35:24\n7R\x03SURWHFW\x03WKH\x03ULJKWV\x03RI\x03WKH\x03DXWKRU\x0bV\x0c\x03DQG\x03SXEOLVKHU\x03ZH\x03LQIRUP\x03 \\RX\x03WKDW\x03WKLV\x033')\x03LV\x03DQ\x03XQFRUUHFWHG\x03SURRI\x03IRU\x03LQWHUQDO\x03EXVLQHVV \x03XVH\x03RQO\\\x03E\\\x03WKH\x03DXWKRU\x0bV\x0c\x0f\x03HGLWRU\x0bV\x0c\x0f\x03UHYL

### Create Vector Store and Retriever

In [96]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
# embedding vectors save in the vectorstore

In [97]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# when user ask question, retrever gives appropriate chunks from vector store.

### Define Prompt Template

In [98]:
from langchain_core.prompts import ChatPromptTemplate

# Define the system prompt
system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"  # this context come from retriever
    # best chunks for the asked question from the vector space.
    # we can define number of chunks that need to come here from retriever
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), # 2 roles, system and human
        ("human", "{input}"), # user input
    ]
)

# after that final prompt created

In [99]:
prompt
# according to the user asked question, context will vary.

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

### Create Retrieval-Augmented Generation (RAG) Chain

In [100]:
# the rag chain contains QA chain and retriever

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt) # create QA chain using llm and prompt
# prompt = system prompt + user input
# system prompt = instruction + context

# Create the RAG chain containing QA chain and retriever
rag_chain = create_retrieval_chain(retriever, qa_chain)
# QA chain has to give answers to questions that ask by user

### Invoke RAG Chain with Example Questions

In [101]:
response = rag_chain.invoke({"input": "what is efficientNet ?"}) # ask questions
response["answer"]

'EfficientNet is a Convolutional Neural Network (CNN) architecture and scaling technique that scales the network breadth, depth, and resolution uniformly using a set of predetermined scaling factors. It is designed to efficiently optimize the model parameters for better performance in image recognition and dataset categorization tasks. The EfficientNet model is often used in transfer learning to improve the performance of CNN models on specific tasks.'

In [102]:
response = rag_chain.invoke({"input": "what is Deep learning"})
response["answer"]

'Deep learning is a subset of artificial intelligence and machine learning that uses neural networks with many layers to learn and make decisions from large amounts of data. It is particularly useful for tasks such as image and speech recognition, natural language processing, and other complex problems that require sophisticated pattern recognition.'

In [103]:
response = rag_chain.invoke({"input": "what is CNN"})
response["answer"]

'CNN stands for Convolutional Neural Network. It is a type of deep learning algorithm commonly used in image processing and computer vision tasks. CNNs are designed to automatically and efficiently extract features from input images, making them extremely useful for tasks such as image segmentation, pattern recognition, and classification. CNNs have been shown to be effective in various applications, including medical image processing, where they can be used to diagnose diseases like diabetic retinopathy, cataract, or glaucoma.'

In [104]:
response = rag_chain.invoke({"input": "What is RAG architecture"})
response["answer"]

"I'm sorry, but I don't have information on RAG architecture in the context provided."

In [105]:
response = rag_chain.invoke({"input": "Can u list down evaluation metrices"})
response["answer"]

'The evaluation metrics commonly used in machine learning and information retrieval are Precision, Recall, and F1-score.'

# Add Chat History

Create a retriever that aware history

In [106]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Define the contextualize system prompt
contextualize_system_prompt = (
    "using chat history and the latest user question, just reformulate question if needed and otherwise return it as is"
)

# Create the contextualize prompt template
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt
)

#### Create History-Aware RAG Chain

In [107]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an intelligent chatbot. Use the following context to answer the question. If you don't know the answer, just say that you don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

prompt

# now => ChatPromptTemplate(input_variables=['chat_history', 'context', 'input']

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.

In [108]:
# Create the question-answering chain
qa_chain = create_stuff_documents_chain(llm, prompt)

# Create the history aware RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

#### Manage Chat Session History

In [109]:
# different user has different history
# here we use session id. according to session id, store the history in a list

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the store for session histories (a dictionary)
store = {}

# Function to get the session history for a given session ID
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the conversational RAG chain with session history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# we can save the dictionary in jason format, save in a database.

#### Invoke Conversational RAG Chain with Example Questions

In [116]:
response = conversational_rag_chain.invoke(
    {"input": "what is diabitic retinopathy ?"},
    config={"configurable": {"session_id": "101"}},  # this is session id.
)   # each user have unique session id
response["answer"]

"Diabetic retinopathy is a diabetes complication that affects the eyes. It's caused by damage to the blood vessels of the light-sensitive tissue at the back of the eye (retina). In the early stages of diabetic retinopathy, there may be no symptoms. As the condition progresses, it can lead to vision loss and blindness. Diabetic retinopathy can be classified into two main types: non-proliferative diabetic retinopathy (NPDR) and proliferative diabetic retinopathy (PDR). Regular eye exams and early detection are crucial in managing diabetic retinopathy and preventing vision loss."

In [111]:
response = conversational_rag_chain.invoke(
    {"input": "what is Eye disease classification ?"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

'Eye disease classification refers to the process of using deep learning techniques, such as Convolutional Neural Networks (CNNs) and transfer learning, to analyze images of eyes and classify them into different categories based on the presence or absence of specific eye diseases. Researchers use image data mining and machine learning algorithms to develop models that can automatically detect and classify various eye diseases, such as diabetic retinopathy, glaucoma, and macular edema, from eye images. Early detection of eye diseases through automated classification methods can help in timely intervention and treatment, ultimately improving patient outcomes.'

In [117]:
response = conversational_rag_chain.invoke(
    {"input": "eye diseases"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"There are various eye diseases that can affect the eyes and vision. Some common eye diseases include:\n\n1. Cataracts: Clouding of the lens in the eye, leading to blurry vision.\n2. Glaucoma: Damage to the optic nerve often caused by high intraocular pressure.\n3. Age-related macular degeneration (AMD): Degeneration of the macula, leading to central vision loss.\n4. Diabetic retinopathy: Damage to the blood vessels in the retina due to diabetes.\n5. Retinal detachment: Separation of the retina from the underlying layers of the eye.\n6. Conjunctivitis: Inflammation of the conjunctiva, causing redness and irritation.\n7. Dry eye syndrome: Insufficient tear production or poor tear quality, leading to dryness and discomfort.\n8. Keratoconus: Progressive thinning and bulging of the cornea, causing distorted vision.\n9. Retinitis pigmentosa: Inherited disorder causing progressive vision loss.\n10. Uveitis: Inflammation of the uvea, the middle layer of the eye.\n\nThese are just a few exampl

In [119]:
response = conversational_rag_chain.invoke(
    {"input": "can you list down eye diseases"},
    config={"configurable": {"session_id": "101"}},
)
response["answer"]

"Certainly! Here is a list of some common eye diseases:\n\n1. Cataracts\n2. Glaucoma\n3. Age-related macular degeneration (AMD)\n4. Diabetic retinopathy\n5. Retinal detachment\n6. Conjunctivitis\n7. Dry eye syndrome\n8. Keratoconus\n9. Retinitis pigmentosa\n10. Uveitis\n\nThese are just a few examples of eye diseases. It's important to consult with an eye care professional for proper diagnosis and treatment if you suspect you have any eye-related issues."

In [120]:
response = conversational_rag_chain.invoke(
    {"input": "explain "},
    config={"configurable": {"session_id": "101"}},  # previous session id
)
response["answer"]

"Of course! Here is a brief explanation of each of the listed eye diseases:\n\n1. Cataracts: Cataracts are characterized by the clouding of the lens in the eye, leading to blurry vision and eventual vision loss if left untreated. Surgery is often required to remove the cloudy lens and replace it with an artificial one.\n\n2. Glaucoma: Glaucoma is a group of eye conditions that damage the optic nerve, usually due to high intraocular pressure. If untreated, it can lead to permanent vision loss. Treatment typically involves eye drops, medication, or surgery.\n\n3. Age-related macular degeneration (AMD): AMD is a progressive eye disease that affects the macula, leading to central vision loss. There are two types: dry AMD (gradual breakdown of the macula) and wet AMD (abnormal blood vessel growth under the retina).\n\n4. Diabetic retinopathy: Diabetic retinopathy is a complication of diabetes that damages the blood vessels in the retina, leading to vision problems and potential blindness. M

In [121]:
response = conversational_rag_chain.invoke(
    {"input": "explain more"},
    config={"configurable": {"session_id": "102"}},  # for different session id
)
response["answer"]

"I'm sorry, but it seems like the context provided is related to ophthalmology abbreviations, clinical observations, and examination techniques. If you have a specific question or need clarification on a particular topic within this context, please let me know so I can provide you with more detailed information."

In [122]:
response = conversational_rag_chain.invoke(
    {"input": "My eyes feel dry and irritated all the time. Could this be a sign of something serious?"},
    config={"configurable": {"session_id": "102"}},
)
response["answer"]

"Feeling dry and irritated eyes could be a sign of a condition called dry eye syndrome. Dry eye syndrome is a common condition that occurs when your tears are not able to provide adequate lubrication for your eyes. It can be caused by various factors such as aging, certain medications, environmental factors, or underlying health conditions.\n\nWhile dry eye syndrome is usually not a serious condition, it can cause discomfort and affect your quality of life. It's essential to consult an eye care professional for a proper evaluation and diagnosis. They can recommend appropriate treatments such as artificial tears, prescription eye drops, or other therapies to help manage your symptoms and improve eye comfort."

In [123]:
response = conversational_rag_chain.invoke(
    {"input": "I notice my vision is cloudy, especially in bright light. Could I have cataracts?"},
    config={"configurable": {"session_id": "102"}},
)
response["answer"]

'Cloudy vision, especially in bright light, can be a symptom of cataracts. Cataracts are a common age-related condition where the lens of the eye becomes cloudy, leading to blurred or cloudy vision. Other symptoms of cataracts may include difficulty seeing at night, sensitivity to light, seeing halos around lights, and faded colors.\n\nIf you are experiencing cloudy vision, especially in bright light, it is essential to consult an eye care professional for a comprehensive eye examination. They can assess your symptoms, perform tests to diagnose cataracts, and discuss treatment options if necessary. Cataract surgery is a common and effective treatment for cataracts, where the cloudy lens is removed and replaced with an artificial lens to restore clear vision.'

In [124]:
response = conversational_rag_chain.invoke(
    {"input": "What are some good habits for maintaining healthy eyes?"},
    config={"configurable": {"session_id": "102"}},
)
response["answer"]

'Maintaining healthy eyes is essential for overall well-being. Here are some good habits to help keep your eyes healthy:\n\n1. **Regular Eye Exams:** Schedule routine eye exams with an eye care professional to monitor your eye health and detect any issues early.\n\n2. **Healthy Diet:** Eat a balanced diet rich in fruits, vegetables, and omega-3 fatty acids to support eye health.\n\n3. **Protective Eyewear:** Wear sunglasses that block UV rays and safety glasses when engaging in activities that could pose a risk to your eyes.\n\n4. **Proper Lighting:** Ensure adequate lighting when reading or working on a computer to reduce eye strain.\n\n5. **Take Breaks:** Follow the 20-20-20 rule - every 20 minutes, look at something 20 feet away for at least 20 seconds to reduce eye strain.\n\n6. **Stay Hydrated:** Drink plenty of water to maintain overall hydration, including eye moisture.\n\n7. **Quit Smoking:** Smoking can increase the risk of eye diseases such as cataracts and macular degenerati

In [126]:
response = conversational_rag_chain.invoke(
    {"input": "Give a list with points"},
    config={"configurable": {"session_id": "102"}},
)
response["answer"]

'Certainly! Here is a list of good habits for maintaining healthy eyes:\n\n1. Schedule regular eye exams.\n2. Eat a balanced diet rich in fruits, vegetables, and omega-3 fatty acids.\n3. Wear protective eyewear, such as sunglasses and safety glasses.\n4. Ensure proper lighting when reading or using screens.\n5. Take breaks and follow the 20-20-20 rule to reduce eye strain.\n6. Stay hydrated by drinking plenty of water.\n7. Quit smoking to reduce the risk of eye diseases.\n8. Manage screen time and practice good habits like blinking regularly.\n9. Exercise regularly to improve blood circulation.\n10. Get 7-8 hours of quality sleep each night.\n\nFollowing these habits can help you maintain healthy eyes and overall eye health.'

In [128]:
response = conversational_rag_chain.invoke(
    {"input": "What are the symptoms of depression?"},
    config={"configurable": {"session_id": "103"}},
)
response["answer"]

"I'm sorry, but the information provided in the context does not mention symptoms of depression. If you have any other questions or need information on a different topic, feel free to ask!"

In [131]:
response = conversational_rag_chain.invoke(
    {"input": "What are the symptoms of a fungal skin infection?"},
    config={"configurable": {"session_id": "104"}},
)
response["answer"]

'Symptoms of a fungal skin infection can include itching, redness, scaling, and sometimes blistering. In some cases, there may be a maculopapular rash present. It is important to note that clinical signs alone may not definitively distinguish between bacterial and fungal skin infections.'

In [132]:
response = conversational_rag_chain.invoke(
    {"input": "What is my name"},
    config={"configurable": {"session_id": "104"}},
)
response["answer"]

"I'm sorry, but I don't have access to your personal information, so I don't know your name."

In [133]:
response = conversational_rag_chain.invoke(
    {"input": "What is Deep Learning?"},
    config={"configurable": {"session_id": "105"}},
)
response["answer"]

'Deep learning is a type of machine learning that uses artificial neural networks to model and solve complex problems. It involves training algorithms to learn from large amounts of data to make predictions or decisions without being explicitly programmed to do so. Deep learning is particularly useful for tasks such as image and speech recognition, natural language processing, and medical diagnosis.'